In [82]:
import pandas as pd
from pandasql import sqldf
import numpy as np
import h3
from geopy.geocoders import Nominatim

In [83]:
data = pd.read_csv("FlightDistanceTest.csv")

Checking the data information, i.e. the data types, column names, and the number of non-null values.

In [84]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Normalised City Pair  3000 non-null   object 
 1   Departure Code        3000 non-null   object 
 2   Arrival Code          3000 non-null   object 
 3   Departure_lat         3000 non-null   float64
 4   Departure_lon         3000 non-null   float64
 5   Arrival_lat           3000 non-null   float64
 6   Arrival_lon           3000 non-null   float64
dtypes: float64(4), object(3)
memory usage: 164.2+ KB


In [85]:
data.head(10)

,Normalised City Pair,Departure Code,Arrival Code,Departure_lat,Departure_lon,Arrival_lat,Arrival_lon
0,"London, United Kingdom - New York, United States Of America",LHR,JFK,51.5,-0.45,40.64,-73.79
1,"Johannesburg, South Africa - London, United Kingdom",JNB,LHR,-26.1,28.23,51.47,-0.45
2,"London, United Kingdom - New York, United States Of America",LHR,JFK,51.5,-0.45,40.64,-73.79
3,"Johannesburg, South Africa - London, United Kingdom",JNB,LHR,-26.1,28.23,51.47,-0.45
4,"London, United Kingdom - Singapore, Singapore",SIN,LHR,1.3,103.98,51.47,-0.45
5,"London, United Kingdom - New York, United States Of America",JFK,LCY,40.6,-73.79,51.50,0.05
6,"London, United Kingdom - New York, United States Of America",LCY,JFK,51.5,0.05,40.64,-73.79
7,"London, United Kingdom - Newark, United States Of America",EWR,LHR,40.7,-74.18,51.47,-0.45
8,"Bombay, India - London, United Kingdom",LHR,BOM,51.5,-0.45,19.09,72.86
9,"Dubai, United Arab Emirates - London, United Kingdom",DXB,LHR,25.3,55.35,51.47,-0.45


Appending latitude and longitude into a single master list in order to calculate the distances between departure and arrival coordinates.

In [86]:
def coordinates_departure() -> list:
    cords = []
    for latitude, longitude in zip(data['Departure_lat'], data['Departure_lon']):
        cords.append([latitude, longitude])
    return cords
    
def coordinates_arrival() -> list:
    cords = []
    for latitude, longitude in zip(data['Arrival_lat'], data['Arrival_lon']):
        cords.append([latitude, longitude])
    return cords

Calculating route distances in miles based on latitude and longitude.

In [87]:
def distance():
    departure = coordinates_departure()
    arrival = coordinates_arrival()

    distances_miles = []
    for cord1, cord2 in zip(departure, arrival):
        distance_km = h3.point_dist(cord1, cord2, unit='km')
        distance_miles = distance_km / 1.609344
        distances_miles.append(distance_miles)
    
    return distances_miles

Appending the calculated distances to the dataframe.

In [88]:
distances = distance()
data['Distance'] = distances
data['Distance'] = round(data['Distance'], 2)

In [89]:
data

,Normalised City Pair,Departure Code,Arrival Code,Departure_lat,Departure_lon,Arrival_lat,Arrival_lon,Distance
0,"London, United Kingdom - New York, United States Of America",LHR,JFK,51.5,-0.45,40.64,-73.79,3442.49
1,"Johannesburg, South Africa - London, United Kingdom",JNB,LHR,-26.1,28.23,51.47,-0.45,5635.89
2,"London, United Kingdom - New York, United States Of America",LHR,JFK,51.5,-0.45,40.64,-73.79,3442.49
3,"Johannesburg, South Africa - London, United Kingdom",JNB,LHR,-26.1,28.23,51.47,-0.45,5635.89
4,"London, United Kingdom - Singapore, Singapore",SIN,LHR,1.3,103.98,51.47,-0.45,6764.24
...,...,...,...,...,...,...,...,...
2995,"New York, United States Of America - Toronto, Canada",LGA,YYZ,40.8,-73.88,43.68,-79.61,354.13
2996,"Newark, United States Of America - Vail/eagle, United States Of America",EGE,EWR,41.3,-85.28,40.69,-74.18,579.99
2997,"Isle Of Man, United Kingdom - Southampton, United Kingdom",SOU,IOM,51.0,-1.36,54.09,-4.63,253.83
2998,"Ottawa, Canada - Toronto, Canada",YYZ,YOW,43.7,-79.61,45.32,-75.67,224.05


In [90]:
pd.set_option('display.max_colwidth', 300)

At this point, my intention was to split the strings from the 'Normalized City Pair' column to create new columns for the city of departure, city of arrival, country of departure, and country of arrival. The goal was to verify the exact addresses for each coordinate and to slice strings in the 'Normalized City Pair' column to check if any of the words matched the generated address. After using the *geopy* library to find the exact addresses of the coordinates, I quickly realized that this approach was impractical. This was because some addresses use different character sets (e.g. Arabic letters) or the exact address does not include the city name. For example, the exact address for the coordinates '-26.1, 28.23' is 'Kosmos Road, Ekurhuleni Ward 16, Kempton Park, City of Ekurhuleni Metropolitan Municipality, Gauteng, 1627, South Africa.' The corresponding row in the 'Normalized City Pair' column for that location is 'Johannesburg,' which my method would not have detected since 'Johannesburg' is not mentioned in the exact address *geopy* provided.

In [91]:
def location(latitude_and_longitude: str) -> str: # returns location (exact address) of the input coordinates
    geolocator = Nominatim(user_agent="locator")
    geolocation = geolocator.reverse(latitude_and_longitude)
    return geolocation.address if geolocation else ""
print(location("-26.1, 28.23")) # example

Kosmos Road, Ekurhuleni Ward 16, Kempton Park, City of Ekurhuleni Metropolitan Municipality, Gauteng, 1627, South Africa


Since "Normalizsed City Pair' column has been removed from the scope of my analysis, another potential anchor point could be to verify if airport codes are consistent for each set of geographic coordinates. First, I will create a temporary table that will store concatenated values of latitude and longitude (for easier interpretation), along with the airport codes for both departure and arrival. This will enable me to search for data inconsistencies using SQL.

In [92]:
# concatenating longitude and latitude into a single column
data["departure_coords"] = data["Departure_lat"].astype(str) + " " + data["Departure_lon"].astype(str)
data["arrival_coords"] = data["Arrival_lat"].astype(str) + " " + data["Arrival_lon"].astype(str)

In [93]:
departure_code = pd.DataFrame(data["Departure Code"].copy())
arrival_code = pd.DataFrame(data["Arrival Code"].copy())

temp_df = departure_code.join(data["departure_coords"], how='inner')
temp_df = temp_df.join(arrival_code)
temp_df = temp_df.join(data["arrival_coords"], how='inner')
temp_df.columns = ["departure_code", "departure_coords", "arrival_code", "arrival_coords"] # renaming columns for easier handling in sql

In [94]:
temp_df

,departure_code,departure_coords,arrival_code,arrival_coords
0,LHR,51.5 -0.45,JFK,40.64 -73.79
1,JNB,-26.1 28.23,LHR,51.47 -0.45
2,LHR,51.5 -0.45,JFK,40.64 -73.79
3,JNB,-26.1 28.23,LHR,51.47 -0.45
4,SIN,1.3 103.98,LHR,51.47 -0.45
...,...,...,...,...
2995,LGA,40.8 -73.88,YYZ,43.68 -79.61
2996,EGE,41.3 -85.28,EWR,40.69 -74.18
2997,SOU,51.0 -1.36,IOM,54.09 -4.63
2998,YYZ,43.7 -79.61,YOW,45.32 -75.67


Now, I will proceed with querying the above table to determine if there are any coordinates that are associated with more than one airport code (which of course - shouldn't occur).

In [95]:
sqldf(""" 
    SELECT departure_coords, COUNT(DISTINCT departure_code) AS count_of_distinct_departure_codes
    FROM temp_df
    GROUP BY departure_coords
    HAVING COUNT(DISTINCT departure_code) > 1;
""")

,departure_coords,count_of_distinct_departure_codes
0,-1.3 36.81,5
1,-12.0 -77.11,2
2,-20.0 23.43,2
3,-20.4 57.68,2
4,-23.4 -46.48,3
5,-26.1 28.23,9
6,-33.9 151.23,2
7,-37.7 144.85,2
8,1.3 103.98,10
9,13.7 100.77,3


As we can see, some coordinates are associated with multiple airport codes. Coordinates '51.5, -0.45' have the largest number of airport codes assigned to them. Let's proceed and check the specific codes matched with these coordinates.

In [96]:
sqldf("""
    SELECT DISTINCT departure_code
    FROM temp_df
    WHERE departure_coords LIKE '51.5 -0.45'
""")

,departure_code
0,LHR
1,MAA
2,JFK
3,BOM
4,EWR
5,MAN
6,DXB
7,SIN
8,MAD
9,DEL


There are **39 unique** airport codes associated with the coordinates '51.5, -0.45'. We can use the location function mentioned earlier to decode these coordinates and determine the correct code that should be assigned to them. Addtionally, we can verify any airport code by searching for 'airport codes list' on Google for example, and using any of the resulting website.

In [97]:
location("51.5 -0.45")

'Stockley Road, West Drayton, London Borough of Hillingdon, London, Greater London, England, UB7 9BN, United Kingdom'

The coordinates '51.5, -0.45' correspond to London. Therefore, any other airport codes associated with these specific coordinates are, in fact, errors in the dataset.

We can also explore the specific airport codes assigned to the highlighted coordinates.

In [98]:
sqldf("""
    SELECT departure_coords, GROUP_CONCAT(DISTINCT departure_code) AS departure_codes
    FROM (
        SELECT DISTINCT departure_coords, departure_code
        FROM temp_df
    ) unique_codes
    GROUP BY departure_coords
    HAVING COUNT(DISTINCT departure_code) > 1
""")

,departure_coords,departure_codes
0,-1.3 36.81,"NBO,LHR,EBB,JNB,ACC"
1,-12.0 -77.11,"LIM,EZE"
2,-20.0 23.43,"MUB,GBE"
3,-20.4 57.68,"MRU,SEZ"
4,-23.4 -46.48,"GRU,LHR,JFK"
5,-26.1 28.23,"JNB,CAI,GBE,DXB,HKG,DUR,GRU,LHR,DOH"
6,-33.9 151.23,"SYD,NRT"
7,-37.7 144.85,"MEL,LHR"
8,1.3 103.98,"SIN,LHR,MNL,EWR,CGK,AKL,NRT,JFK,KUL,FRA"
9,13.7 100.77,"BKK,HKG,FRA"


Now, I will query the columns containing information about arrival coordinates and arrival airport codes.

In [99]:
sqldf(""" 
    SELECT arrival_coords, COUNT(DISTINCT arrival_code) AS count_distinct_arrival_codes
    FROM temp_df
    GROUP BY arrival_coords
    HAVING COUNT(DISTINCT arrival_code) > 1;
""")

,arrival_coords,count_distinct_arrival_codes
0,40.07 116.58,2
1,45.47 9.18,2
2,49.01 2.54,2
3,51.47 -0.45,2


There are significantly fewer errors in the arrival coordinates and codes. However, some errors do still exist. Let's examine all of the airport codes assigned to these coordinates.

In [100]:
sqldf("""
    SELECT arrival_coords,
    GROUP_CONCAT(DISTINCT arrival_code) as arrival_code
    FROM (
        SELECT DISTINCT arrival_coords, arrival_code
        FROM temp_df
    ) unique_codes
    GROUP BY arrival_coords
    HAVING COUNT(DISTINCT arrival_code) > 1
""")


,arrival_coords,arrival_code
0,40.07 116.58,"PEK,BJS"
1,45.47 9.18,"LIN,MXP"
2,49.01 2.54,"CDG,PAR"
3,51.47 -0.45,"LHR,LON"


Considering that various airport codes are assigned to the same geographic coordinates, it can be inferred that the distances calculated for these airports are incorrect. The distances were calculated based on coordinates provided by the International Air Transport Association. Therefore, errors are present in the 'Departure Code' and 'Arrival Code' columns.

In [101]:
data.to_excel("results.xlsx", index=False)

### Useful Links and References:
* [Getting distance between two points based on latitude/longitude](https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude)
* [Airport codes](https://www.ccra.com/airport-codes/)
* [Get address from given coordinate using python](https://stackoverflow.com/questions/60928516/get-address-from-given-coordinate-using-python)
* [How to Use SQL in pandas Using pandasql Queries](https://www.datacamp.com/tutorial/how-to-use-sql-in-pandas-using-pandasql-queries)